<H1 style="text-align: center">EEEM066 - Fundamentals of Machine Learning</H1>
<H1 style="text-align: center">Week - 8: Deep system design (I)</H1>

> Dr. Xiatian (Eddy) Zhu, Dr Syed Sameed Husain

> xiatian.zhu@surrey.ac.uk, sameed.husain@surrey.ac.uk

**Introduction**

This lab is about the designing a Resnet based deep learning system for classification task.

For this tutorial, we will use the CIFAR10 dataset. It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

Happy Programming!⚡⚡

# Contents

<!-- **Training an image classifier**
We will do the following steps in order:

1) Load and normalize the CIFAR10 training and test datasets using torchvision

2) Define a Resnet architecture

3) Define a loss function

4) Train the network on the training data

5) Test the network on the test data -->

*   [Load and normalize CIFAR10](https://colab.research.google.com/drive/1aBEIdwhC_7TFlPpaAeWTRwbeTGjZT3V0#scrollTo=FlBp3a3PYQjB)
*   [Design a Resnet System](https://colab.research.google.com/drive/1aBEIdwhC_7TFlPpaAeWTRwbeTGjZT3V0#scrollTo=GV5WisJhYQjC)
*   [Define a Loss function and optimizer](https://colab.research.google.com/drive/1aBEIdwhC_7TFlPpaAeWTRwbeTGjZT3V0#scrollTo=zJ4E8YvkYQjD)
*   [Train the network on the training data](https://colab.research.google.com/drive/1aBEIdwhC_7TFlPpaAeWTRwbeTGjZT3V0#scrollTo=4XFeKdU3YQjE)
*   [Test the network on the test data](https://colab.research.google.com/drive/1aBEIdwhC_7TFlPpaAeWTRwbeTGjZT3V0#scrollTo=Ncv2PSfCYQjE)


**Python libraries required**
1. Numpy
4. torch
5. torchvision

## Load and normalize CIFAR10

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 50028118.83it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


## Design a Resnet System

In [3]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [4]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

## Define a Loss function and optimizer

Let’s use a Classification Cross-Entropy loss and SGD with momentum.

In [5]:
learning_rate=0.005
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

## Train the network on the training data

We simply have to loop over our data iterator, and feed the inputs to the network and optimize.

In [6]:
num_epochs=10
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 5 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/10], Step [100/391] Loss: 1.5512
Epoch [1/10], Step [200/391] Loss: 1.4568
Epoch [1/10], Step [300/391] Loss: 1.4272
Epoch [2/10], Step [100/391] Loss: 1.1992
Epoch [2/10], Step [200/391] Loss: 0.9688
Epoch [2/10], Step [300/391] Loss: 1.0585
Epoch [3/10], Step [100/391] Loss: 0.8400
Epoch [3/10], Step [200/391] Loss: 0.7869
Epoch [3/10], Step [300/391] Loss: 0.7109
Epoch [4/10], Step [100/391] Loss: 0.7630
Epoch [4/10], Step [200/391] Loss: 0.7258
Epoch [4/10], Step [300/391] Loss: 0.6135
Epoch [5/10], Step [100/391] Loss: 0.6413
Epoch [5/10], Step [200/391] Loss: 0.6867
Epoch [5/10], Step [300/391] Loss: 0.6019
Epoch [6/10], Step [100/391] Loss: 0.3655
Epoch [6/10], Step [200/391] Loss: 0.6073
Epoch [6/10], Step [300/391] Loss: 0.5267
Epoch [7/10], Step [100/391] Loss: 0.6164
Epoch [7/10], Step [200/391] Loss: 0.4763
Epoch [7/10], Step [300/391] Loss: 0.3668
Epoch [8/10], Step [100/391] Loss: 0.3796
Epoch [8/10], Step [200/391] Loss: 0.4443
Epoch [8/10], Step [300/391] Loss:

## Test the network on the test data

We have trained the network. But we need to check the network performance on the test dataset

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.pt')

Accuracy of the model on the test images: 82.6 %
